In [1]:
import torch
import numpy as np
import os
#import torchani
import h5py
import random
import math
import sys
import parmed as pmd
from parmed.amber import Rst7



In [35]:
periodic_table = {'H':1,
                   'HA':1,
                   'HB2':1,
                   'HB3':1,
                   'He':2,
                   'Li':3,
                   'Be':4,
                   'B':5,
                   'C':6,
                   'CA':6,
                   'CB':6,
                   'N':7,
                   'O':8,
                   'F':9,
                   'Ne':10,
                   'Na':11,
                   'Mg':12,
                   'Al':13,
                   'Si':14,
                   'P':15,
                   'S':16,
                   'SG':16,
                   'Cl':17,
                   'Ar':18}

polarizabilities = {'H':4.50711,
                   'HA':4.50711,
                   'HB2':4.50711,
                   'HB3':4.50711,
                   'He':1.38375,
                   'Li':164.1125,
                   'Be':37.74,
                   'B':20.5,
                   'C':11.3,
                   'CA':11.3,
                   'CB':11.3,
                   'N':7.4,
                   'O':5.3,
                   'F':3.74,
                   'Ne':2.6611,
                   'Na':162.7,
                   'Mg':71.2,
                   'Al':57.8,
                   'Si':37.3,
                   'P':25,
                   'S':19.4,
                   'SG':19.4,
                   'Cl':14.6,
                   'Ar':11.083} #https://doi.org/10.1080/00268976.2018.1535143

def XYZtoTensor(XYZ):
    #abro el archivo
    with open(XYZ, 'r') as f:
        #cuento las lineas del archivo
        line_count = 0
        for i, line in enumerate(f):
            if i == 0:
            #me quedo con la cantidad de atomos y hago listas vacias de especies y coordenadas
                species = [0]*int(line)
                coordinates = [[0,0,0]]*int(line)
            line_count += 1
            if i in range(2, len(species)+2):
                species[i-2] = line.split()[0]
        f.close()

        frames = math.floor(line_count / (len(species) + 2))
        #genero una lista vacia con la cantidad de frames que hay
        tensor_coordinates = [0]*frames
        tensor_species = [0]*frames

        #transformo los atomos en sus indices de la tabla periodica
        for i, atom in enumerate(species):
            species[i] = periodic_table[atom]

        #lo convierto en un tensor de la dimension requerida
        species = torch.tensor(species).unsqueeze(0)

        #recorro las lineas de nuevo :s
    with open(XYZ, 'r') as f:
        for i, line in enumerate(f):
            for numero_de_frame, frame in enumerate(tensor):

                #genero una lista con los elementos de cada linea
                lista_linea = line.split()
                #guardo las coordenadas
                coordinates[i] = lista_linea[-3:]
                print(coordinates)
            frame = np.array(coordinates, dtype=float)

        #transformo las coordenadas a tensores de pytorch
        tensor = torch.tensor(tensor, requires_grad=True, dtype=torch.float32)

    return tensor_species, tensor_coordinates


def TensortoXYZ(species, tensor, nombre):
    XYZ = open (nombre,'w')
    num_atoms = len(species[0])
    frames = tensor.shape[0]
    symbols  = [] #vector de los simbolos de los atomos
    for i, atom in enumerate(species[0]):
        symbols.append(list(periodic_table.keys())[list(periodic_table.values()).index(atom)])
    for i in range(frames):
        XYZ.write(num_atoms+'\n'+'archivo generado con TensortoXYZ'+'\n')
        for j in range(num_atoms):
            line = symbols[j]+" "+str(float(tensor[i][j][0]))+" "+str(float(tensor[i][j][1]))+" "+str(float(tensor[i][j][2]))+"\n"
            XYZ.write(line)
    XYZ.close()
    return

def extraer_N_estructuras(lista_indices, formula, database):
    coordenadas = database[formula+'/coordinates']
    estructuras_seleccionadas = []
    for i in lista_indices:
        estructuras_seleccionadas.append(coordenadas[i])
    estructuras_sel = torch.tensor(estructuras_seleccionadas)
    return estructuras_sel

def extraer_N_cargas(indice, formula, database):

    cargas_cm5        = database[formula+'/wb97x_dz.cm5_charges'][indice]
    cargas_hirshfeld = database[formula+'/wb97x_dz.hirshfeld_charges'][indice]
    cargas_mbis       = database[formula+'/wb97x_tz.mbis_charges'][indice]

    return cargas_cm5, cargas_hirshfeld, cargas_mbis


def unit_vector(vector):
    """ Returns the unit vector of the vector."""
    return vector / np.linalg.norm(vector)


def angle_between(v1, v2):
    """Finds angle between two vectors"""
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def x_rotation(vector,theta):
    """Rotates 3-D vector around x-axis"""
    R = np.array([[1,0,0],[0,np.cos(theta),-np.sin(theta)],[0, np.sin(theta), np.cos(theta)]])
    return np.dot(R,vector)

def y_rotation(vector,theta):
    """Rotates 3-D vector around y-axis"""
    R = np.array([[np.cos(theta),0,np.sin(theta)],[0,1,0],[-np.sin(theta), 0, np.cos(theta)]])
    return np.dot(R,vector)

def z_rotation(vector,theta):
    """Rotates 3-D vector around z-axis"""
    R = np.array([[np.cos(theta), -np.sin(theta),0],[np.sin(theta), np.cos(theta),0],[0,0,1]])
    return np.dot(R,vector)

def makewaters(inflim,suplim,VdWdist):
    x = np.arange(inflim, suplim, VdWdist)
    waters=[]
    for i in range(len(x)):
        for j in range(len(x)):
            for k in range(len(x)):
                pos = (x[i],x[j],x[k])
                waters.append(pos)
    waters=np.array(waters)
    return waters

def mergezandcoordXYZ(symbols,coord,waters):
    print(len(symbols)+len(waters))
    print("Molecule solvated")
    for i in range(0,len(simbolos)):
        print(symbols[i],coord[i][0].item(),coord[i][1].item(),coord[i][2].item())
    for i in range(0,int(len(waters)/3)):
        print('O ',waters[3*i][0].item(),waters[3*i][1].item(),waters[3*i][2].item())
        print('H ',waters[(3*i)+1][0].item(),waters[(3*i)+1][1].item(),waters[(3*i)+1][2].item())
        print('H ',waters[(3*i)+2][0].item(),waters[(3*i)+2][1].item(),waters[(3*i)+2][2].item())

def deleteOverlaped(waters,coord,VdW):
    newwaters=[]
    for i in range(0,len(waters)):
        tooclose=False
        for j in range(0,len(coord)):
          x=coord[j][0]-waters[i][0]
          y=coord[j][1]-waters[i][1]
          z=coord[j][2]-waters[i][2]
          dist=math.sqrt(x**2+y**2+z**2)
          if (dist < VdW):
            tooclose=True
        if not tooclose:
            newwaters.append(waters[i])
    newwaters=np.array(newwaters)
    return newwaters

def keepOnlyNCloser(waters,coord,N):
    if N >= len(waters):
        return waters
    watersWindex=[]
    for i in range(0,len(waters)):
        x0=coord[0][0]-waters[0][0]
        y0=coord[0][1]-waters[0][1]
        z0=coord[0][2]-waters[0][2]
        distMIN=math.sqrt(x0**2+y0**2+z0**2)
        for j in range(0,len(coord)):
            x=coord[j][0]-waters[i][0]
            y=coord[j][1]-waters[i][1]
            z=coord[j][2]-waters[i][2]
            dist=math.sqrt(x**2+y**2+z**2)
            if dist<distMIN:
               distMIN=dist
        vec=[waters[i][0],waters[i][1],waters[i][2],distMIN]
        watersWindex.append(vec)
    watersWindex=np.array(watersWindex)
    watersWindex=watersWindex[watersWindex[:, 3].argsort()]
    newwaters=[]
    for i in range(0,N):
        vec=(watersWindex[i][0],watersWindex[i][1],watersWindex[i][2])
        newwaters.append(vec)
    newwaters=np.array(newwaters)
    return newwaters

def randnormalvec():
    vec=np.array([(0.5-random.random()),(0.5-random.random()),(0.5-random.random())])
    norm=math.sqrt(vec[0]**2+vec[1]**2+vec[2]**2)
    return vec/norm

def addHydrogens(waters):
    theta_WAT=109
    newwaters=[]
    for i in range(0,len(waters)):
        r=0.94
        vecH1=r*randnormalvec()
        vecH2=r*randnormalvec()

        x1, y1, z1 = vecH1
        r1=math.sqrt(x1**2+y1**2+z1**2)
        phi1=math.atan(y1/x1)
        theta1=math.acos(z1/r1)

        x2, y2, z2 = vecH2
        r2=math.sqrt(x2**2+y2**2+z2**2)
        phi2=math.atan(y2/x2)
        theta2=math.acos(z2/r2)
        #print(np.degrees(theta2))
        # print("ANTES",np.degrees(angle_between(vecH1,vecH2)))
        if (x1>0):
            rotx=-phi1
            roty=-theta1
        else:
            rotx=-phi1+np.radians(180)
            roty=-theta1

        vecH1=z_rotation(vecH1,rotx)
        vecH2=z_rotation(vecH2,rotx)
        vecH1=y_rotation(vecH1,roty)
        vecH2=y_rotation(vecH2,roty)

        theta2=np.radians(theta_WAT)
        x2=r2*math.sin(theta2)*math.cos(phi2)
        y2=r2*math.sin(theta2)*math.sin(phi2)
        z2=r2*math.cos(theta2)
        vecH2 = x2, y2, z2

        vecH1=z_rotation(vecH1,-rotx)
        vecH2=z_rotation(vecH2,-rotx)
        vecH1=y_rotation(vecH1,-roty)
        vecH2=y_rotation(vecH2,-roty)

        vecH1=vecH1+waters[i]
        vecH2=vecH2+waters[i]

        # check=True
        # if (abs(np.degrees(angle_between(vecH1-waters[i],vecH2-waters[i]))-109)>0.01):
        #     check=check and False

        newwaters.append(waters[i])
        newwaters.append(vecH1)
        newwaters.append(vecH2)
    newwaters=np.array(newwaters)

    return newwaters

def solvate(structure, wat):
    waters = makewaters(-30,30,3.1)
    waters = deleteOverlaped(waters, structure, 4)
    waters = keepOnlyNCloser(waters, structure, wat)
    waters = addHydrogens(waters)
    waters = torch.Tensor(waters)
    solvated_structure = torch.stack((structure, waters))

    return solvated_structure

def append_water_charges(charges, wat):
    charges = charges.numpy()
    charges_h20 = np.array([-0.834,0.417,0.417]*wat)
    total_charges = np.concatenate((charges, charges_h20))
    total_charges= torch.Tensor(total_charges)
    return total_charges


def dist(x1,y1,z1,x2,y2,z2):
    import math
    dist_cuad = (x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2
    dist = math.sqrt(dist_cuad)
    return dist

def E_field2(r,q,nqm,ntot,i):
    E_field=torch.zeros(3)
    for j in range(nqm,ntot):
        rij = np.array([r[j][0].item() - r[i][0].item(), r[j][1].item() - r[i][1].item(), r[j][2].item() - r[i][2].item()])
        distij = dist(r[i][0].item(), r[i][1].item(),r[i][2].item(), r[j][0].item(), r[j][1].item(),r[j][2].item())
        rij = rij/distij
        for k in range(3):
            E_field[k] = E_field[k]+(q[j].item()/distij**2)*rij[k]
    E_field2=E_field[0]**2+E_field[1]**2+E_field[2]**2
    return E_field2

def estaEnRango(r,nqm,j,cutoff):
    esta=False
    i=0
    while (not esta and i<nqm):
        rij = np.array([r[j][0].item() - r[i][0].item(), r[j][1].item() - r[i][1].item(), r[j][2].item() - r[i][2].item()])
        distij = dist(r[i][0].item(), r[i][1].item(),r[i][2].item(), r[j][0].item(), r[j][1].item(),r[j][2].item())
        esta = esta or distij < cutoff
        i=i+1
    return esta

def get_indice_aguas_en_rango(r,atoms_qm,cutoff):
    lista=[]
    for i in range(0,atoms_qm):
        lista.append(0)
    for i in range(atoms_qm,len(r)):
        if (estaEnRango(r,atoms_qm,i,cutoff)):
            lista.append(1)
        else:
            lista.append(0)
    return lista


def compute_coulomb(r, q_mbis, species, lista, polarizabilities):
    nqm = len(species)
    ntot = len(q_mbis)
 #   E_elec_cm5 = 0.0
 #   E_elec_hir = 0.0
    E_elec_mbis = 0.0
 #   E_elec_mul_sol = 0.0
 #   E_elec_mul_vac = 0.0
    E_pol = 0.0
    for i in range(nqm):
 #       qi_cm5 = q_cm5[i].item()
 #       qi_hir = q_hir[i].item()
        qi_mbis = q_mbis[i].item()
 #       qi_mul_sol = q_mul_sol[i].item()
 #       qi_mul_vac = q_mul_vac[i].item()
        alpha = 0.14818471*polarizabilities[i]
        E_field=torch.zeros(3)
        for j in range(nqm, ntot):
            qj  = q_mbis[j].item()
            rij = np.array([r[j][0].item() - r[i][0].item(), r[j][1].item() - r[i][1].item(), r[j][2].item() - r[i][2].item()])
            distij = dist(r[i][0].item(), r[i][1].item(),r[i][2].item(), r[j][0].item(), r[j][1].item(),r[j][2].item())
            rij = rij/distij
            #if estaEnRango(r,nqm,j,cutoff):
            if lista[j]==1:
 #               E_elec_cm5 = E_elec_cm5 + qi_cm5*qj/distij
 #               E_elec_hir = E_elec_hir + qi_hir*qj/distij
                E_elec_mbis = E_elec_mbis + qi_mbis*qj/distij
 #               E_elec_mul_sol = E_elec_mul_sol + qi_mul_sol*qj/distij
 #               E_elec_mul_vac = E_elec_mul_vac + qi_mul_vac*qj/distij
                for k in range(3):
                    E_field[k] = E_field[k]+(qj/distij**2)*rij[k]
        E_field2 = E_field[0]**2+E_field[1]**2+E_field[2]**2
        E_pol = E_pol + alpha*E_field2
  #  E_elec_cm5 = E_elec_cm5*0.529177249*627.509391
  #  E_elec_hir = E_elec_hir*0.529177249*627.509391
    E_elec_mbis = E_elec_mbis*0.529177249*627.509391
  #  E_elec_mul_sol = E_elec_mul_sol*0.529177249*627.509391
  #  E_elec_mul_vac = E_elec_mul_vac*0.529177249*627.509391
    E_pol = -0.5*E_pol.item()*0.529177249*627.509391

    return E_elec_mbis, E_pol

In [36]:
#----------------------------------------------------------------------------------------
#CARGO LISTA DE ESTRUCTURAS
#----------------------------------------------------------------------------------------
dicZ=[("H",1),("He",2),("Li",3),("Be",4),("B",5),("C",6),("N",7),("O",8),("F",9),("Ne",10)]
database = h5py.File('/home/jsg/ani1x-release.h5', 'r')
#database = h5py.File('/home/jota/BasesDeDatos/ani1x-release.h5', 'r')


#os.remove("energias.txt")
#file_object = open('energias.txt', 'a')
#file_object.write('formula indice E_ref E_elec_cm5 E_elec_hir E_elec_mbis E_elec_mul_sol E_elec_mul_vac E_pol \n')
#file_object.close()

#lista = np.loadtxt('lista.txt', dtype=str)
#lista = lista[254:]
#nombre_archivos = []

#for i in range(len(lista)): #iria len(lista)
#    nombre_archivos.append(lista[i][0]+'_'+lista[i][1])

#for numero, nombre_archivo in enumerate(nombre_archivos):

In [67]:
#----------------------------------------------------------------------------------------
#EXTRAIGO LAS CARGAS DE LA BASE DE DATOS
#----------------------------------------------------------------------------------------
species = database['C3H7N1O3/atomic_numbers']
atoms_qm =  species.shape[0]
formula = 'C3H7N1O3'
indice = 3272

  #  q_cm5 = torch.Tensor(database[formula+'/wb97x_dz.cm5_charges'][indice])
  #  q_hir = torch.Tensor(database[formula+'/wb97x_dz.hirshfeld_charges'][indice])
q_mbis = torch.Tensor(database[formula+'/wb97x_tz.mbis_charges'][indice])



In [68]:
#----------------------------------------------------------------------------------------
#CARGO LOS .RST7 DE LAS ESTRUCTURAS SOLVATADAS
#----------------------------------------------------------------------------------------
r = pmd.load_file('C3H7N1O3/C3H7N1O3_'+str(indice)+'_sol_sp.rst7').coordinates[0]



In [69]:
polarizabilities_948 = [8.126021185, 7.94186892, 8.00772902, 2.05554108, 2.12462337, 2.45993538, 2.0194109,
                    2.316879105, 1.49660198, 1.253177885, 7.847840985, 4.73816177, 5.32171176, 5.84980524]

polarizabilities_1583 = [8.04940293, 7.736854335, 7.07501304, 1.99895848, 1.82239476, 2.094874675, 2.007246675,
                         1.979948315, 1.83698792, 1.3949197, 7.101308425, 5.62306714, 4.80015766, 5.033260135]
polarizabilities_985 = [7.581804490000001, 9.23531016, 10.053369804999999, 1.86160316, 2.3674078400000003, 
                        2.7755080100000002, 3.722896055, 2.194286135, 1.71062871, 3.61978798, 8.11754003, 5.05445525, 6.451949385000001,7.18678446]
polarizabilities_1285 = [6.3652573, 7.6844391100000005, 8.525428094999999, 2.082392665, 1.829648915, 2.23800594, 1.8762743,
                         2.29033865, 2.244191185,2.159117275, 7.831399285, 5.82571559, 5.265369865, 5.157332395]

polarizabilities_497 = [7.753836455000001, 7.03140731, 8.8207232, 2.093527825, 2.4182745, 2.093612975, 2.2052216600000003, 2.214378825, 
                        2.14512038,1.76353295, 8.898406959999999, 5.5734226499999995, 5.39785346,6.26744072]


polarizabilities_552 = [8.06, 8.30, 8.64, 1.50, 2.34, 1.89, 2.02, 2.75, 1.89, 2.32, 6.95, 5.85, 4.25, 6.18]

polarizabilities_1145 = [9.78, 7.80, 9.63, 2.59, 2.39,3.68,1.85,2.85,1.63,2.54,7.84,6.45,6.34,6.70]

polarizabilities_1503 = [8.37, 8.04,7.37,1.62,1.76,1.87,2.37,2.25,1.68,1.26,6.81,5.92,4.18,4.86]

polarizabilities_3100 = [8.05, 7.50,7.44,1.67,1.90,2.05,1.97,2.12,1.82,1.87,6.67,5.22,5.36,5.43]

polarizabilities_3272 = [8.36,7.79,7.53,1.86,2.21,2.27,2.11,2.16,1.87,1.44,6.97,5.36,5.78,5.12]

In [10]:
#----------------------------------------------------------------------------------------
#EXTRAIGO LAS CARGAS MULLIKEN DE LIO
#----------------------------------------------------------------------------------------
    q_mul_sol = []
    inicio = 7 + atoms_qm
    fin = 8 + 2*atoms_qm
    with open(nombre_archivo+'_sol_MK', 'r') as f:
        for renglon, line in enumerate(f):
            if renglon > inicio and renglon < fin:
                q = line.split()[-1]
                q_mul_sol.append(float(q))
    q_mul_vac = []
    with open(nombre_archivo+'_vac_MK', 'r') as f:
        for renglon, line in enumerate(f):
            if renglon > inicio and renglon < fin:
                q = line.split()[-1]
                q_mul_vac.append(float(q))

#-------------------------------------------------

IndentationError: unexpected indent (<ipython-input-10-7478fcf8207c>, line 4)

In [70]:
nombre_archivo = formula+'_'+str(indice)

In [71]:
#----------------------------------------------------------------------------------------
#ORDENO EL TENSOR DE COORDENADAS PARA QUE COINCIDA CON EL ORDEN DE LAS CARGAS
#----------------------------------------------------------------------------------------

#----------------------------------------------------------------------------------------
#ORDENO EL TENSOR DE CARGAS MULLIKEN
#----------------------------------------------------------------------------------------


nombretop=formula+'/'+nombre_archivo+"_vac_ord.prmtop" #revisar
topfile = pmd.load_file(nombretop)
atoms_qm=len(topfile.residues[0])
listZ_top=[]
listS_top=[]
for i in range(0,atoms_qm):
    listZ_top.append(topfile.atoms[i].atomic_number)
    listS_top.append(dicZ[listZ_top[i]-1][0])


listS_ord=sorted(listS_top)
listZ_ord=[]
for i in range(0,len(listS_ord)):
    for j in range(0,len(dicZ)):
        if (dicZ[j][0]==listS_ord[i]):
            listZ_ord.append(dicZ[j][1])

a=''.join(listS_ord)

q_mul_sol_ord = []
q_mul_vac_ord = []

rst7file=Rst7(formula+'/'+nombre_archivo+'_sol_sp.rst7')

coordenadas=rst7file.coordinates[:1][0]
coordQM=coordenadas[0:atoms_qm]

coordQM_ord=[]

for i in range(0,len(listZ_ord)):
    j=0
    encontre=False
    while (j<len(listZ_top) and not encontre):
        if (listZ_top[j]==listZ_ord[i]):
#            q_mul_sol_ord.append(q_mul_sol[j])
#            q_mul_vac_ord.append(q_mul_vac[j])
#            q_mul_sol = np.delete(q_mul_sol, j, axis=0)
#            q_mul_vac = np.delete(q_mul_vac, j, axis=0)

            coordQM_ord.append(coordQM[j])
            coordQM = np.delete(coordQM, j,axis=0)

            listZ_top = np.delete(listZ_top, j,axis=0)
            encontre = True
        j=j+1

    # cargar en un tensor las coordenadas ordenadas

coordQM_ord = torch.Tensor(np.array(coordQM_ord))
coordMM = torch.Tensor(coordenadas[atoms_qm:])
r = torch.cat((coordQM_ord, coordMM))


#    q_mul_sol = torch.Tensor(q_mul_sol_ord)
#    q_mul_vac = torch.Tensor(q_mul_vac_ord)


In [72]:

#----------------------------------------------------------------------------------------
#AGREGO LAS CARGAS DE LAS AGUAS
#----------------------------------------------------------------------------------------
# el número de aguas es un tercio de la diferencia entre la cantidad total de átomos y los del soluto
wat = int((r.shape[0] -  atoms_qm)/3)
#q_cm5     = append_water_charges(q_cm5, wat)
#q_hir    = append_water_charges(q_hir, wat)
q_mbis    = append_water_charges(q_mbis, wat)
#q_mul_vac = append_water_charges(q_mul_vac, wat)
#q_mul_sol = append_water_charges(q_mul_sol, wat)


#genero un array de dimension len(r) que tiene 0 o 1, segun el agua este o no en rango.
#para los atomosqm tiene un 0 pero eso no se usa nunca
indice_a=get_indice_aguas_en_rango(r,atoms_qm,cutoff=15)


In [73]:

#----------------------------------------------------------------------------------------
#CALCULO LA ENERGIA QMMM DEL EMBEBIMIENTO
#----------------------------------------------------------------------------------------
E_elec_mbis, E_pol = compute_coulomb(r, q_mbis, species, indice_a, polarizabilities_3272)


In [74]:
E_elec_mbis+E_pol

-50.135542137093225

In [75]:
E_pol

-8.2517684504595

In [ ]:
#----------------------------------------------------------------------------------------
#EXTRAIGO LA ENERGIA QMMM DE REFERENCIA
#----------------------------------------------------------------------------------------
    with open(nombre_archivo+'_sol_sp.out')as f:
        for line in f:
            if 'QM-MM elec.' in line:
                linea = line.split(" ")
                E_ref = float(linea[-1][:-2])*627.509391